In [35]:
import  os
import  re
import  glob

import  pandas as pd
import  numpy  as np

from src.utils  import *
from src.models import TransE, rTransE

from src.env   import Env
from src.agent import DQN_Network, process_agent_samples

In [ ]:
def load_clg_zfiles(pathfilename):
    
    #nt_file = open(pathfilename,'r')
    lines =[]
    with gzip.open(pathfilename,'r') as fin: 
        all_lines = fin.readline()
        lines= all_lines.decode('utf8').split(' .')
    
    num_lines = 0
    clg_dict  = dict({'s':[],'o':[],'p':[]})
    
    for i,line in enumerate(lines):
        num_lines+=1
        entities = line.split(' ')
        if len(entities) != 3: continue
        clg_dict['s'].append(entities[0])
        clg_dict['p'].append(entities[1])
        clg_dict['o'].append(entities[2])
        
    df = pd.DataFrame(clg_dict)
        
    #print(num_lines)
    return df

In [27]:
path      = 'datasets/clg/clg_full/'
train_file= 'clg_full-train.nt.gz'
test_file = 'clg_full-test.nt.gz'

In [28]:
df_train= load_clg_zfiles(path+train_file)

In [29]:
df_test = load_clg_zfiles(path+test_file)

In [19]:
max_test_batch_size=-1

### Subclass Relation

In [ ]:
print('Running...', train_file, test_file)


data_subclass_train = df_train[df_train['p']== '<http://www.w3.org/2000/01/rdf-schema#subClassOf>']
data_subclass_train = data_subclass_train[['s','o']].rename(columns={'s':'subClass','o':'class'})   
transitive_classes= pd.merge(data_subclass_train,data_subclass_train,how='left',right_on=['subClass'],left_on=['class']).dropna(subset=['class_y'])
del transitive_classes['class_x']
transitive_classes.columns = ['class_0', 'class_1', 'class_2']
transitive_classes = transitive_classes.drop_duplicates(subset=['class_0', 'class_1', 'class_2']) # drop duplicates
data_subclass_train_quads = transitive_classes.reset_index(drop=True)

data_subclass_test = df_test[df_test['p']== '<http://www.w3.org/2000/01/rdf-schema#subClassOf>']
data_subclass_test= data_subclass_test[['s','o']].rename(columns={'s':'subClass','o':'class'})

res = prepare_subclass_data(data_subclass_train,data_subclass_train_quads)
ode_dict, node_count, train_trips, train_quads = res
_, _, test_trips, test_quads = prepare_subclass_data(data_subclass_test)
print(len(train_trips),len(train_quads))

# Train TransE
print('')
model_e  = TransE(node_count,1)
model_e._train(train_trips,train_quads);
model_e._eval(test_trips[:max_test_batch_size])  # evaluate TransE

# Train rTransE
model_r  = rTransE(node_count,1)
model_r._train(train_trips,train_quads,num_epoches=300);
model_r._eval(test_trips[:max_test_batch_size]) # evaluate rTransE

print()

### Asertation Relation

In [ ]:
print('Running...', train_file, test_file)

data_subclass_train = df_train[df_train['p']== '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>']
data_subclass_train = data_subclass_train[['s','o']].rename(columns={'s':'type','o':'class'})   
transitive_classes= pd.merge(data_subclass_train,data_subclass_train,
                             how='left',right_on=['type'],
                             left_on=['class']).dropna(subset=['class_y'])
del transitive_classes['class_x']


transitive_classes.columns = ['type_0', 'class_0', 'class_1']
transitive_classes = transitive_classes.drop_duplicates(subset=['type_0', 'class_0', 'class_1']) # drop duplicates
data_subclass_train_quads = transitive_classes.reset_index(drop=True)

res = prepare_subclass_data(data_subclass_train,data_subclass_train_quads,
                            tc1='class',
                            tc2='type',
                            qc1='type_0',
                            qc2='class_0',
                            qc3='class_1')
node_dict, node_count, train_trips, train_quads = res

data_type_test = df_test[df_test['p']== '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>']
data_type_test= data_type_test[['s','o']].rename(columns={'s':'type','o':'class'})

_, _, test_trips, test_quads = prepare_subclass_data(data_type_test,tc1='class',
                                tc2='type')

print(len(train_trips),len(train_quads))

# tarin TransE
print('')
model_TransE  = TransE(node_count,1)
model_TransE._train(train_trips,train_quads);

model_TransE._eval(test_trips) # evaluate TransE

# train rTransE
model_rTransE  = rTransE(node_count,1)
model_rTransE._train(train_trips,train_quads,num_epoches=300);
model_rTransE._eval(test_trips[:max_test_batch_size])  # evaluate RTransE

print()

### Combined

In [ ]:
print('Running...', train_file, test_file)

data_subclass_train = df_train[df_train['p']== '<http://www.w3.org/2000/01/rdf-schema#subClassOf>']
data_subclass_train = data_subclass_train[['s','o']].rename(columns={'s':'subClass','o':'class'})   
transitive_classes= pd.merge(data_subclass_train,data_subclass_train,how='left',
                             right_on=['subClass'],left_on=['class']).dropna(subset=['class_y'])
del transitive_classes['class_x']
transitive_classes.columns = ['class_0', 'class_1', 'class_2']
transitive_classes = transitive_classes.drop_duplicates(subset=['class_0', 'class_1', 'class_2']) # drop duplicates
data_subclass_train_quads = transitive_classes.reset_index(drop=True)

data_type_train = df_train[df_train['p']== '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>']
data_type_train = data_type_train[['s','o']].rename(columns={'s':'type','o':'class'})   

transitive_classes = pd.merge(data_type_train,data_subclass_train,
                                 how='left',left_on=['class'],right_on=['subClass']).dropna(subset=['class_y'])
del transitive_classes['class_x']
transitive_classes.columns = ['type', 'class_0', 'class_1']
data_cross_quads = transitive_classes.reset_index(drop=True)

res = prepare_crossclass_data(data_subclass_train,data_subclass_train_quads,0,
                        data_type_train,1,
                       transitive_classes,tc1='class',tc2='subClass',
                          ac1='class',ac2='type',
                          qc1='class_0',qc2='class_1',qc3='class_2',
                          cc1='type',cc2='class_0',cc3='class_1')

node_dict, node_count, train_trips, train_quads = res


data_subclass_test = df_test[df_test['p']== '<http://www.w3.org/2000/01/rdf-schema#subClassOf>']
data_subclass_test= data_subclass_test[['s','o']].rename(columns={'s':'subClass','o':'class'})

res = prepare_subclass_data(data_subclass_train,data_subclass_train_quads)
_, _, test_trips1, test_quads1 = res

data_type_test = df_test[df_test['p']== '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>']
data_type_test= data_type_test[['s','o']].rename(columns={'s':'type','o':'class'})

_, _, test_trips2, test_quads2 = prepare_subclass_data(data_type_test,tc1='class',
                                tc2='type')


test_trips = test_trips1 + test_trips2


print(len(train_trips),len(test_trips))


# tarin TransE
print('')
model_TransE  = TransE(node_count,2)
model_TransE._train(train_trips,train_quads);
model_TransE._eval(test_trips[:max_test_batch_size]) # evaluate TransE

# train rTransE
model_rTransE  = rTransE(node_count,2)
model_rTransE._train(train_trips,train_quads,num_epoches=200);
model_rTransE._eval(test_trips[:max_test_batch_size])  # evaluate RTransE


env   = Env(train_trips)
agent = DQN_Network([60, 64, 2],lr=1e-3)
agent_samples = agent.train(env,
        model_rTransE.entity_embds.detach().numpy(),
        model_rTransE.rel_embds.detach().numpy(),
        episodes = 20000)

unique_agent_samples = process_agent_samples(train_quads,agent_samples)
updated_train_trips  = train_trips+unique_agent_samples

update_train_trips = train_trips+unique_agent_samples
model_rTransE._train(update_train_trips,train_quads,num_epoches=100);
model_rTransE._eval(test_trips[:max_test_batch_size])  # evaluate aTransE

print()

env   = Env(train_trips)
agent = DQN_Network([60, 64, 2],lr=1e-3)
agent_samples = agent.train(env,
        model_rTransE.entity_embds.detach().numpy(),
        model_rTransE.rel_embds.detach().numpy(),
        episodes = 20000)

unique_agent_samples = process_agent_samples(train_quads,agent_samples)
updated_train_trips  = train_trips+unique_agent_samples

update_train_trips = train_trips+unique_agent_samples
model_rTransE._train(update_train_trips,train_quads,num_epoches=100);
model_rTransE._eval(test_trips[:max_test_batch_size])  # evaluate aTransE

print()
